# Strong&Weak Core

v0.5 判断一种分配是不是 Strong Core

Search for the results in traditional way

In [190]:
from itertools import combinations
import csv

def output(A):
    '''
    输出矩阵
    '''
    for i in range(len(A)):
        for j in range(len(A[i])):
            print(A[i][j], end = '    ')
        print('\n')
    for i in range((len(A[0])-1)*5+1):
        print('-', end = '')
    print('\n')
    return None

def inpt(file):
    '''
    file is the name of the inputed csv file
    '''
    C = []
    # 从文件中读取数据
    with open(file) as c_file:
        c_reader = csv.reader(c_file, delimiter = ' ')
        for row in c_reader:
            C.append(row)

    for i in range(len(C)):
        for j in range(len(C[i])):
            C[i][j] = int(C[i][j])
    
    return C

def empty(n, m):
    '''
    创建 n*m 的零矩阵
    '''
    A = []
    for i in range (n):
        A.append([])
        for j in range(m):
            A[i].append(0)
    return A


# assignments
Mu1 = inpt('1_Mu1.txt')
Mu2 = inpt('1_Mu2.txt')
Sigma = inpt('1_Sigma.txt')

# Preferences
P = inpt('1_P.txt')
# C is the co-ownership structure
C = inpt('1_C.txt')
n_player = len(C)
I = []
for i in range(n_player):
    I.append(i+1)
n_object = len(C[0])
O = []
for a in range(n_object):
    O.append(a+1)

## Notifications

1. To decide whether a nonempty coalition $C$  can weakly blocks assignment $\mu$ via assignment $\sigma$


In [191]:
print('Preference:')
output(P)
print('Omega:')
output(C)
print('Sigma:')
output(Sigma)
print('Mu_1:')
output(Mu1)
print('Mu_2:')
output(Mu2)

Preference:
2    1    

2    1    

2    1    

------

Omega:
1    1    

0    0    

0    0    

------

Sigma:
1    0    

0    0    

0    1    

------

Mu_1:
1    0    

0    1    

0    0    

------

Mu_2:
1    0    

0    0    

0    0    

------



In [221]:
def assign(i, S):
    '''
    输入 assignment S 和个体，返回在该分配下，他所分配的物品
    返回零代表该分配下他什么都没得到
    '''
    a = 0
    for aa in range(len(S[0])):
        if S[i-1][aa]:
            a = aa+1
    return a

def R_c(i, a, b):
    '''
    判断对i来说，是否偏好a
    '''
    if a>0:
        ua = P[i-1][a-1]
    else:
        ua = 0
    if b>0:
        ub = P[i-1][b-1]
    else:
        ub = 0
        
    if ua>=ub:
        return True
    else:
        return False


def P_c(i, a, b):
    '''
    判断对i来说，是否严格偏好a
    '''
    if a>0:
        ua = P[i-1][a-1]
    else:
        ua = 0
    if b>0:
        ub = P[i-1][b-1]
    else:
        ub = 0
        
    if ua>ub:
        return True
    else:
        return False

def private_own(coalition, Omega):
    '''
    to find if a coalition is feasible via co-owenership structure Omega
    '''
    A = []
    for a in O:
        k = 1
        for i in I:
            if Omega[i-1][a-1] and not (i in coalition):
                k = 0
                break
        if k:
            A.append(a)
    return A

def transpose(A):
    '''
    transpose a matrix
    '''
    n = len(A)
    m = len(A[0])
    ans = empty(m, n)
    for i in range(m):
        for j in range(n):
            ans[i][j] = A[j][i]
    return ans
    
    
def weakly_block(coalition, M, S):
    '''
    check is assignment Mu is weakly blocked by coalition via Sigma
    '''
    k1 = 1
    k2 = 0
    for i in coalition:
        if not R_c(i, assign(i, S), assign(i, M)):
            k1 = 0
    for i in coalition:
        if P_c(i, assign(i, S), assign(i, M)):
            k2 = 1
    k3 = 0
    if set(private_own(coalition, S))<=set(private_own(coalition, C)):
        k3 = 1
    
    if k1 and k2 and k3:
        return True
    else:
        return False

def strongly_block(coalition, M, S):
    '''
    check is assignment Mu is strongly blocked by coalition via Sigma
    '''
    k1 = 1
    for i in coalition:
        if not P_c(i, assign(i, S), assign(i, M)):
            k1 = 0
    k3 = 0
    if set(private_own(coalition, S))<=set(private_own(coalition, C)):
        k3 = 1
    
    if k1 and k3:
        return True
    else:
        return False    
    

def if_in_strong_core(Mu):
    
    # check if the strong core is empty
    # 对coalition进行组合
    COALITION = []

    for popu in range(1, n_player):
        for coalition in combinations([1,2,3], popu+1):
            COALITION.append(list(coalition))
            
    ans = True
    I_plus = [0]+I
    for i in I_plus:
        T1 = []
        for j in I:
            if j==i:
                T1.append(1)
            else:
                T1.append(0)
        for k in I_plus:
            T2 = []
            for m in I:
                if m==k:
                    T2.append(1)
                else:
                    T2.append(0)
            T = transpose([T1, T2])
            for coalition in COALITION:
                if weakly_block(coalition, Mu, T):
                    #print('Coalition: '+str(coalition)+'\n')
                    #output(T)
                    ans = False
    return ans

def if_in_weak_core(Mu):
    
    # check if the strong core is empty
    # 对coalition进行组合
    COALITION = []

    for popu in range(1, n_player):
        for coalition in combinations([1,2,3], popu+1):
            COALITION.append(list(coalition))
            
    ans = True
    I_plus = [0]+I
    for i in I_plus:
        T1 = []
        for j in I:
            if j==i:
                T1.append(1)
            else:
                T1.append(0)
        for k in I_plus:
            T2 = []
            for m in I:
                if m==k:
                    T2.append(1)
                else:
                    T2.append(0)
            T = transpose([T1, T2])
            for coalition in COALITION:
                if strongly_block(coalition, Mu, T):
                    #print('Coalition: '+str(coalition)+'\n')
                    #output(T)
                    ans = False
    return ans


In [217]:
coalition = [1, 3]
weakly_block(coalition, Mu1, Sigma)
strongly_block(coalition, Mu1, Sigma)

False

## To find if the strong-core is empty

In [228]:
index = 0
I_plus = [0]+I
for i in I_plus:
    T1 = []
    for j in I:
        if j==i:
            T1.append(1)
        else:
            T1.append(0)
    for k in I_plus:
        T2 = []
        for m in I:
            if m==k:
                T2.append(1)
            else:
                T2.append(0)
        T = transpose([T1, T2])
        if if_in_strong_core(T):
            index += 1
            output(T)
if not index:
    print('Strong core is empty')

Strong core is empty


## To find if the weak-core is empty

In [229]:
index = 0
I_plus = [0]+I
for i in I_plus:
    T1 = []
    for j in I:
        if j==i:
            T1.append(1)
        else:
            T1.append(0)
    for k in I_plus:
        T2 = []
        for m in I:
            if m==k:
                T2.append(1)
            else:
                T2.append(0)
        T = transpose([T1, T2])
        if if_in_weak_core(T):
            index += 1
            if index==1:
                print('The assignments in the weak core are:\n')
            output(T)
if not index:
    print('Strong core is empty')

The assignments in the weak core are:

1    0    

0    0    

0    0    

------

1    0    

0    1    

0    0    

------

1    0    

0    0    

0    1    

------



In [143]:
a = 0
b = 0
from itertools import product
for i in product([0,1], repeat = 2):
    for j in product([0,1], repeat = 2):
        for k in product([0,1], repeat = 2):
            T = [list(i),list(j),list(k)]
            a +=1
            #for coalition in COALITION:
            if weakly_block([1,3], Mu1, T):
                b +=1
                
                #output(T)

24